In [ ]:
from tqdm import tqdm_notebook


# 셀레늄 import

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time



chromedriver = '크롬 드라이브 디렉토리' 

'''
# headless chrome 옵션임
headless_options = webdriver.ChromeOptions()
headless_options.add_argument('headless')
driver = webdriver.Chrome(chromedriver, options=headless_options)  # 대소문자 주의

# 추가 옵션들
headless_options.add_argument('window-size=1920x1080')
headless_options.add_argument('disable-gpu')
headless_options.add_argument('lang=ko_KR')
headless_options.add_argument('User-Agent:  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Safari/537.36')
'''

In [ ]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://m.sacticket.co.kr/msac/exhi/search_list.do")
time.sleep(1)
elem = driver.find_element_by_id('s_program_name')
#container > section > fieldset
elem.send_keys('교향악축제')
elem.send_keys(Keys.RETURN)
# 서치버튼
select_click("#container > section > fieldset > button", driver)
time.sleep(5)

#과거 종료 프로그램탭
select_click("#container > section > article > ul > li:nth-child(2) > a", driver)
time.sleep(2)

# 프로그램 갯수 
program_qty = driver.find_element_by_css_selector(f"#container > section > article > ul > li:nth-child(2) > a > span").text
program_qty = int(program_qty)

# 옆으로 가기 버튼 클릭해야하는 횟수 계산
next_click_qty = program_qty // 10

# 마지막 페이지에서 몇번 더 돌아야하는가?
final_page_crawl_qty = program_qty % 10


# 상세보기 링크를 돌면서 가져온다

past_program_link_list = []
past_program_name_list = []
count = 0 # 초기화
num = 0 # 초기화
while count < next_click_qty:
    try:
        for num in range(1,11):
            program_link = driver.find_element_by_css_selector(f"#ajaxDiv > ul > li:nth-child({num}) > a").get_attribute('href')
            print(f"{num}번째 항목을 크롤링했삼")
            program_name = driver.find_element_by_css_selector(f"#ajaxDiv > ul > li:nth-child({num}) > a > span.title").text
            
            past_program_link_list.append(program_link)
            past_program_name_list.append(program_name)
        count += 1
        select_click("#ajaxDiv > div > a:nth-child(3) > button", driver)
        print("> 버튼 눌렀습니다")
        time.sleep(1.2)
        
        
    except NoSuchElementException as e:
        print(e)

for num in range(1, final_page_crawl_qty + 1):
    program_link = driver.find_element_by_css_selector(f"#ajaxDiv > ul > li:nth-child({num}) > a").get_attribute('href')
    program_name = driver.find_element_by_css_selector(f"#ajaxDiv > ul > li:nth-child({num}) > a > span.title").text
    print(f"마지막 페이지의 {num}번째 항목을 크롤링했삼")
    past_program_link_list.append(program_link)
    past_program_name_list.append(program_name)


time.sleep(1)
print("상세보기 링크 크롤링이 종료됨")

driver.quit()

print(len(past_program_link_list))
print(len(past_program_name_list))
past_program_name_list


In [ ]:

program_content_list = []

for url in tqdm_notebook(now_df['link']):
    driver.get(url)
    time.sleep(2)
    select_click('#container > section > article > ul > li:nth-child(2) > a', driver)
    time.sleep(1)
    program_content = driver.find_element_by_css_selector("#container > section > article > div.exh_guide > div:nth-child(2)").text
    program_content_list.append(program_content)
    time.sleep(1)


driver.quit()

now_df['contents'] = program_content_list
